## Основы статистики
https://stepik.org/course/76/syllabus
### 2. Сравнение средних
### 2.1 T-распределение

In [20]:
from scipy import stats
import numpy as np

def get_se(sd, N):
    '''Стандартная ошибка'''
    return sd/np.sqrt(N)

def z_stat(X, M, sd, N):
    '''z-значение'''
    se = get_se(sd, N)
    return (X-M)/se

def p_value_norm(z_stat, alternative = 'two-sided'):
    '''p_value для нормального распределения'''
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [21]:
#шаги 6-9

z = z_stat(10.8, 10, 2, 25)
p_value_norm(2, 'two-sided')

0.04550026389635842

In [24]:
#шаги 6-9
#используем t-распределение

def p_value_t(t_stat, N, alternative = 'two-sided'):
    df = N-1 #кол-во степеней свободы
    if alternative == 'two-sided':
        return 2*(1-stats.t.cdf(np.abs(t_stat),df))
    if alternative == 'less':
        return stats.t.cdf(t_stat, df)
    if alternative == 'greater':
        return 1 - stats.t.cdf(t_stat, df)

p_value_t(z, 25)

0.0569398499365914

In [26]:
#шаг 10
p_value_t(-2, 15)

0.06528795288911193